# Sequence classification and generation using LSTMs

Adapted from: Practical PyTorch: Classifying Names with a Character-Level RNN, Generating Names with a Character-Level RNN

# Part 1: Sequence classification

We will be building and training a basic character-level RNN to classify words. A character-level RNN reads words as a series of characters - outputting a prediction and "hidden state" at each step, feeding its previous hidden state into each next step. We take the final prediction to be the output, i.e. which class the word belongs to.

Specifically, we'll train on a few thousand surnames from 18 languages of origin, and predict which language a name is from based on the spelling:

```
$ python predict.py Hinton
(-0.47) Scottish
(-1.52) English
(-3.57) Irish

$ python predict.py Schmidhuber
(-0.19) German
(-2.48) Czech
(-2.68) Dutch
```

# Preparing the Data

Included in the `data/names` directory are 18 text files named as "[Language].txt". Each file contains a bunch of names, one name per line, mostly romanized (but we still need to convert from Unicode to ASCII).

We'll end up with a dictionary of lists of names per language, `{language: [names ...]}`. The generic variables "category" and "line" (for language and name in our case) are used for later extensibility.

In [ ]:
import glob

all_filenames = glob.glob('data/05/names/*.txt')
print(all_filenames)

In [ ]:
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicode_to_ascii('Ślusàrski'))

In [ ]:
# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename).read().strip().split('\n')
    return [unicode_to_ascii(line) for line in lines]

for filename in all_filenames:
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)
print('n_categories =', n_categories)

Now we have `category_lines`, a dictionary mapping each category (language) to a list of lines (names). We also kept track of `all_categories` (just a list of languages) and `n_categories` for later reference.

In [ ]:
print(category_lines['Italian'][:5])

# Turning Names into Tensors

Now that we have all the names organized, we need to turn them into Tensors to make any use of them.

In [ ]:
import torch

letter2idx = dict(zip(list(all_letters), list(range(n_letters))))

def line_to_idxs(line):
    return torch.tensor([letter2idx[letter] for letter in line], dtype=torch.long)

In [ ]:
print(line_to_idxs('Jones'))

# Network architecture

Before autograd, creating a recurrent neural network in Torch involved cloning the parameters of a layer over several timesteps. The layers held hidden state and gradients which are now entirely handled by the graph itself. This means you can implement a RNN in a very "pure" way, as regular feed-forward layers.

<img src="images/05/classifier.png">

In [1]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class LSTMClassifier(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim

        ################# TODO ##################
        
        # Input layer
        self.word_embeddings = None

        # LSTM layer
        self.lstm = None

        # Output layer
        # The linear layer that maps from hidden state space to classes
        self.linear = None
        
        ##########################################
        
        self.init_hidden()
        
    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        self.hidden = None

    def forward(self, sentence):
        ################# TODO ##################
        
        # Input representation
        embeddings = None
        
        # Process sequence using LSTM
        lstm_out, self.hidden = None, None
        
        # Output logits
        logits = None
        
        ##########################################
        
        log_probs = F.log_softmax(logits, dim=1)
        
        return log_probs

# Preparing for Training

Before going into training we should make a few helper functions. The first is to interpret the output of the network, which we know to be a likelihood of each category. We can use `Tensor.topk` to get the index of the greatest value:

In [ ]:
def category_from_output(output):
    top_n, top_i = output.data.topk(1) # Tensor out of Variable with .data
    category_i = top_i[0][0]
    return all_categories[category_i], category_i

print(category_from_output(torch.randn(1, n_categories)))

We will also want a quick way to get a training example (a name and its language):

In [ ]:
import random

def random_training_pair():                                                                                                               
    category = random.choice(all_categories)
    line = random.choice(category_lines[category])
    category_tensor = Variable(torch.LongTensor([all_categories.index(category)]))
    line_tensor = Variable(line_to_idxs(line))
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = random_training_pair()
    print('category =', category, '/ line =', line)

# Training the Network

Now all it takes to train this network is show it a bunch of examples, have it make guesses, and tell it if it's wrong.

Define the model

In [ ]:
n_hidden = 128
model = LSTMClassifier(n_hidden, n_hidden, n_letters, n_categories)

For the [loss function `nn.NLLLoss`](http://pytorch.org/docs/nn.html#nllloss) is appropriate, since the last layer of the RNN is `nn.LogSoftmax`.

In [ ]:
criterion = nn.NLLLoss()

We will also create an "optimizer" which updates the parameters of our model according to its gradients. We will use the vanilla SGD algorithm with a low learning rate.

In [ ]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Each loop of training will:

* Create input and target tensors
* Create a zeroed initial hidden state
* Read each letter in and
    * Keep hidden state for next letter
* Compare final output to target
* Back-propagate
* Return the output and loss

In [ ]:
def train(targets, inputs):
    model.zero_grad()
    model.init_hidden()
    
    output = model(inputs)

    loss = criterion(output, targets)
    loss.backward()

    optimizer.step()

    return output, loss.data.item()

Now we just have to run that with a bunch of examples. Since the `train` function returns both the output and loss we can print its guesses and also keep track of loss for plotting. Since there are 1000s of examples we print only every `print_every` time steps, and take an average of the loss.

In [ ]:
import time
import math

n_epochs = 100000
print_every = 10000
plot_every = 1000

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for epoch in range(1, n_epochs + 1):
    # Get a random training input and target
    category, line, category_tensor, line_tensor = random_training_pair()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print epoch number, loss, name and guess
    if epoch % print_every == 0:
        guess, guess_i = category_from_output(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (epoch, epoch / n_epochs * 100, time_since(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if epoch % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

# Plotting the Results

Plotting the historical loss from `all_losses` shows the network learning:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

plt.figure()
plt.plot(all_losses)

# Evaluating the Results

To see how well the network performs on different categories, we will create a confusion matrix, indicating for every actual language (rows) which language the network guesses (columns). To calculate the confusion matrix a bunch of samples are run through the network with `evaluate()`, which is the same as `train()` minus the backprop.

In [ ]:
# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):

    output = model(line_tensor)
    
    return output[-1].view(1, -1)

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = random_training_pair()
    output = evaluate(line_tensor)
    guess, guess_i = category_from_output(output)
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

plt.show()

You can pick out bright spots off the main axis that show which languages it guesses incorrectly, e.g. Chinese for Korean, and Spanish for Italian. It seems to do very well with Greek, and very poorly with English (perhaps because of overlap with other languages).

# Running on User Input

In [ ]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    output = evaluate(Variable(line_to_idxs(input_line)))

    # Get top N categories
    topv, topi = output.data.topk(n_predictions, 1, True)
    predictions = []

    for i in range(n_predictions):
        value = topv[0][i]
        category_index = topi[0][i]
        print('(%.2f) %s' % (value, all_categories[category_index]))
        predictions.append([value, all_categories[category_index]])

predict('Dovesky')
predict('Jackson')
predict('Satoshi')

# Part 2: Sequence generation

In the previous part, you used RNNs to classify names into their language of origin. In this part you will use an RNN to generate names.

You already have the key ingredients for this part. We will add a few lines of code to the model we used in the previous part for this task. 
* We will add a start of sequence and end of sequence symbols to our vocabulary to indicate the beginning and end of the sequence. 
* We have a new objective function now. We have a loss for every time-step which involves predicting the next character given all the previous characters. 

<img src="images/05/lm.png">

# Training the model

In [ ]:
letter2idx['<SOS>'] = n_letters
letter2idx['<EOS>'] = n_letters + 1

idx2letter = {value: key for key, value in letter2idx.items()}
    
def line_to_idxs(line):
    letter_idxs = [letter2idx[letter] for letter in line]
    letter_idxs = [letter2idx['<SOS>']] + letter_idxs + [letter2idx['<EOS>']]
    return torch.tensor(letter_idxs, dtype=torch.long)

In [ ]:
class NameLM(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, num_classes):
        super(NameLM, self).__init__()
        self.hidden_dim = hidden_dim

        ################# TODO ##################
        
        # Input layer
        self.word_embeddings = None

        # LSTM layer
        self.lstm = None

        # Output layer
        # The linear layer that maps from hidden state space to classes
        self.linear = None
        
        ##########################################
        
        self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        self.hidden = None

    def forward(self, sentence):
        
        ################# TODO ##################
        
        # Input representation
        embeddings = None
        
        # Process sequence using LSTM
        lstm_out, self.hidden = None, None
        
        # Output logits
        logits = None
        
        ##########################################
        
        log_probs = F.log_softmax(logits, dim=1)
        
        return log_probs

In [ ]:
model = NameLM(n_hidden, n_hidden, n_letters+2, n_letters+2)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Keep track of losses for plotting
current_loss = 0
all_losses = []

start = time.time()

for epoch in range(1, n_epochs + 1):
    # Get a random training input and target
    category, line, category_tensor, line_tensor = random_training_pair()
    output, loss = train(line_tensor[1:], line_tensor[:-1])
    current_loss += loss

    # Print epoch number, loss, name and guess
    if epoch % print_every == 0:
        print('%d %d%% (%s) %.4f' % (epoch, epoch / n_epochs * 100, time_since(start), loss))
        
    # Add current loss avg to list of losses
    if epoch % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

# Sampling from the model

To sample we give the network a letter and ask what the next one is, feed that in as the next letter, and repeat until the EOS token.
* Create tensors for input category, starting letter, and empty hidden state
* Create a string output_str with the starting letter
* Up to a maximum output length,
  * Feed the current letter to the network
  * Get the next letter from highest output, and next hidden state
  * If the letter is EOS, stop here
  * If a regular letter, add to output_str and continue
* Return the final name

In [ ]:
max_length = 20

# Generate given a category and starting letter
def generate(start_chars='', temperature=0.5):
    
    model.init_hidden()
    
    start_char = '<SOS>'
    chars_input = letter2idx[start_char]

    output_str = ''
    
    for char in start_chars:
        chars_input = torch.tensor([chars_input])
        output = model(chars_input)
        chars_input = letter2idx[char]
        output_str += char
    
    for i in range(max_length):

        chars_input = torch.tensor([chars_input])
        output = model(chars_input)

        ################### TODO ####################
        # Sample as a multinomial distribution
        output_distribution = None
        sample_char_id = None
        sample_char = None
        #############################################
        
        # Stop at EOS, or add to output_str
        if sample_char == '<EOS>':
            break
        else:
            output_str += sample_char
            chars_input = sample_char_id

    return output_str

In [ ]:
generate()